In [1]:
import xgboost as xgb
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

#### CARGAMOS EL ARCHIVO INSTALLS CON FEATURES SACADOS DEL ARCHIVO INSTALLS

In [73]:
installs = pd.read_csv('TP 1er Cuatrimestre 2019/installsConFeatures.csv', low_memory=False, parse_dates = ['created'], index_col=['created'])
installs = installs.sort_index()
installs.head()

,Unnamed: 0,application_id,ref_type,ref_hash,attributed,implicit,device_countrycode,device_brand,device_model,session_user_agent,...,agentes,top50agentes,top10agentes,kindReducido,top10tipo,top10leng,early_morning,weekend,easter_holiday,nuevoDisositivo
created,,,,,,,,,,,,,,,,,,,,,
2019-04-18 00:00:01.560,94738,70,1,4432995619177048534,False,False,6287817205707153877,0,208,1095,...,0,0,0,1,1,9,0,False,1,1
2019-04-18 00:00:01.851,94737,70,1,5904733559638204455,False,False,6287817205707153877,0,208,1095,...,0,0,0,1,1,9,0,False,1,0
2019-04-18 00:00:05.152,111877,65,1,896373747754111825,False,True,6287817205707153877,6,102,2,...,2,2,1,0,0,8,0,False,1,0
2019-04-18 00:00:05.589,76146,27,0,3399210824535017892,False,False,6287817205707153877,0,190,2,...,21,14,6,1,1,2,0,False,1,1
2019-04-18 00:00:06.795,435184,339,0,1541425881979513687,False,False,6287817205707153877,0,190,2,...,68,46,10,1,1,6,0,False,1,1


### Dropeamos columnas que quedaron mal

In [74]:
installs.drop(['Unnamed: 0'],axis = 1,inplace=True)

#### Luego cargamos la primer ventana y sus lebels correspondientes a la segunda (Asi con todas las demas)

In [75]:
info1 = installs['2019-04-21':'2019-04-23'].reset_index().groupby('ref_hash').agg({'created' : 'min'})
info1['created'] = info1['created'] - pd.Timestamp('2019-04-21 00:00:00')
info1['created'] = info1['created'].transform(lambda x: x.total_seconds())
info1 = info1.reset_index()
info1 = info1.merge(installs['2019-04-18':'2019-04-20'].reset_index(),on="ref_hash", how="right")
info1 = info1.rename(columns={'created_x':'label','created_y':'created'})

label1 = info1['label'].to_frame()
train1 = info1.drop('label', axis=1)

In [76]:
info2 = installs['2019-04-22':'2019-04-24'].reset_index().groupby('ref_hash').agg({'created' : 'min'})
info2['created'] = info2['created'] - pd.Timestamp('2019-04-22 00:00:00')
info2['created'] = info2['created'].transform(lambda x: x.total_seconds())
info2 = info2.reset_index()
info2 = info2.merge(installs['2019-04-19':'2019-04-21'].reset_index(),on="ref_hash", how="right")
info2 = info2.rename(columns={'created_x':'label','created_y':'created'})

label2 = info2['label'].to_frame()
train2 = info2.drop('label', axis=1)

In [77]:
info3 = installs['2019-04-23':'2019-04-25'].reset_index().groupby('ref_hash').agg({'created' : 'min'})
info3['created'] = info3['created'] - pd.Timestamp('2019-04-23 00:00:00')
info3['created'] = info3['created'].transform(lambda x: x.total_seconds())
info3 = info3.reset_index()
info3 = info3.merge(installs['2019-04-20':'2019-04-22'].reset_index(),on="ref_hash", how="right")
info3 = info3.rename(columns={'created_x':'label','created_y':'created'})

label3 = info3['label'].to_frame()
train3 = info3.drop('label', axis=1)

In [78]:
info4 = installs['2019-04-24':'2019-04-26'].reset_index().groupby('ref_hash').agg({'created' : 'min'})
info4['created'] = info4['created'] - pd.Timestamp('2019-04-24 00:00:00')
info4['created'] = info4['created'].transform(lambda x: x.total_seconds())
info4 = info4.reset_index()
info4 = info4.merge(installs['2019-04-21':'2019-04-23'].reset_index(),on="ref_hash", how="right")
info4 = info4.rename(columns={'created_x':'label','created_y':'created'})

label4 = info4['label'].to_frame()
train4 = info4.drop('label', axis=1)

In [79]:
ventana_ultima = installs['2019-04-24':'2019-04-26'].reset_index()

In [80]:
ventana_ultima.head()

,created,application_id,ref_type,ref_hash,attributed,implicit,device_countrycode,device_brand,device_model,session_user_agent,...,agentes,top50agentes,top10agentes,kindReducido,top10tipo,top10leng,early_morning,weekend,easter_holiday,nuevoDisositivo
0,2019-04-24 00:00:00.178,21,1,2105892908739159676,False,False,6287817205707153877,0,74,0,...,0,0,0,1,1,0,0,False,0,1
1,2019-04-24 00:00:02.223,226,1,208059911196607757,False,True,6287817205707153877,6,163,2,...,2,2,1,84,10,8,0,False,0,0
2,2019-04-24 00:00:02.352,70,1,9071467689792743380,False,False,6287817205707153877,0,208,1095,...,0,0,0,1,1,9,0,False,0,0
3,2019-04-24 00:00:02.353,70,1,9071467689792743380,False,True,6287817205707153877,0,208,1095,...,0,0,0,28,6,9,0,False,0,1
4,2019-04-24 00:00:02.764,70,1,7588153492836870321,False,False,6287817205707153877,0,208,1095,...,0,0,0,1,1,9,0,False,0,1


#### Como al algoritmo que aplicaremos despues no le gustan los NAN, a esos nan le ponemos 3 dias ( en segundos )

In [81]:
train1 = train1.dropna()
train2 = train2.dropna()
train3 = train3.dropna()
train4 = train4.dropna()
ventana_ultima = ventana_ultima.dropna()
label1 = label1.fillna(3*24*60*60)
label2 = label2.fillna(3*24*60*60)
label3 = label3.fillna(3*24*60*60)
label4 = label4.fillna(3*24*60*60)


#### Ahora sacaremos mas labels para cada ventana en particular...

In [82]:
train1['tiempoDesdeComienzoDeVentana'] = (installs.reset_index()['created'] - pd.Timestamp('2019-04-18 00:00:00')).transform(lambda x: x.total_seconds())
train2['tiempoDesdeComienzoDeVentana'] = (installs.reset_index()['created'] - pd.Timestamp('2019-04-19 00:00:00')).transform(lambda x: x.total_seconds())
train3['tiempoDesdeComienzoDeVentana'] = (installs.reset_index()['created'] - pd.Timestamp('2019-04-20 00:00:00')).transform(lambda x: x.total_seconds())
train4['tiempoDesdeComienzoDeVentana'] = (installs.reset_index()['created'] - pd.Timestamp('2019-04-21 00:00:00')).transform(lambda x: x.total_seconds())
ventana_ultima['tiempoDesdeComienzoDeVentana'] = (installs.reset_index()['created'] - pd.Timestamp('2019-04-24 00:00:00')).transform(lambda x: x.total_seconds())


In [83]:
train1['accionesDesdeQueComenzoLaVentana'] = np.arange(len(train1))
train2['accionesDesdeQueComenzoLaVentana'] = np.arange(len(train2))
train3['accionesDesdeQueComenzoLaVentana'] = np.arange(len(train3))
train4['accionesDesdeQueComenzoLaVentana'] = np.arange(len(train4))
ventana_ultima['accionesDesdeQueComenzoLaVentana'] = np.arange(len(ventana_ultima))

In [84]:
train1.drop(['created'],axis =1, inplace = True )
train2.drop(['created'],axis =1, inplace = True )
train3.drop(['created'],axis =1, inplace = True )
train4.drop(['created'],axis =1, inplace = True )
ventana_ultima.drop(['created'],axis =1, inplace = True )


In [85]:
#### XGBOOST LE GUSTAN LAS COLUMNAS ORDENADAS... 
ventana_ultima=ventana_ultima.sort_index(axis=1, ascending=False)
train1=train1.sort_index(axis=1, ascending=False)
train2=train2.sort_index(axis=1, ascending=False)
train3=train3.sort_index(axis=1, ascending=False)
train4=train4.sort_index(axis=1, ascending=False)



In [86]:

train1.head()

,wifi,weekend,user_agent,top50marcas,top50agentes,top10tipo,top10marcasSinNan,top10marcas,top10leng,top10apps,...,dia,device_model,device_language,device_countrycode,device_brand,attributed,application_id,agentes,accionesDesdeQueComenzoLaVentana,TopApp/Otro
0,False,False,0,0,0,1,0,0,0,0,...,20,120,0,6287817205707153877,0,False,158,0,0,0
1,False,False,0,0,0,1,0,0,0,0,...,18,70,0,6287817205707153877,0,False,77,0,1,0
2,False,False,68,6,2,1,6,6,8,0,...,18,70,34,6287817205707153877,6,False,302,2,2,0
3,True,False,13403,0,46,1,0,0,3,0,...,20,190,3,6287817205707153877,0,False,339,68,3,0
4,False,False,0,0,0,1,0,0,0,0,...,19,74,0,6287817205707153877,0,False,94,0,4,0


#### Ahora comenzaremos a entrenar con XGBOOST

In [396]:
train1.shape,label1.shape

((160861, 35), (160861, 1))

In [397]:
train1.columns

Index(['wifi', 'weekend', 'user_agent', 'top50marcas', 'top50agentes',
       'top10tipo', 'top10marcasSinNan', 'top10marcas', 'top10leng',
       'top10apps', 'top10agentes', 'tiempoDesdeComienzoDeVentana',
       'session_user_agent', 'ref_type', 'ref_hash', 'nuevoDisositivo',
       'minuto', 'kindReducido', 'kind', 'ip_address', 'implicit', 'hora',
       'easter_holiday', 'early_morning', 'diaDeLaSemana', 'dia',
       'device_model', 'device_language', 'device_countrycode', 'device_brand',
       'attributed', 'application_id', 'agentes',
       'accionesDesdeQueComenzoLaVentana', 'TopApp/Otro'],
      dtype='object')

In [414]:
train1 = train1.drop('wifi',axis=1,inplace=False)
train2 = train2.drop('wifi',axis=1,inplace=False)
train3 = train3.drop('wifi',axis=1,inplace=False)
train4 = train4.drop('wifi',axis=1,inplace=False)
ventana_ultima = ventana_ultima.drop('wifi',axis=1,inplace=False)

In [399]:
data_dmatrix = xgb.DMatrix(data=train,label=label1)


In [444]:
xg_reg1 = xgb.XGBRegressor(objective ='reg:squarederror', 
                colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)

In [484]:
X_train, X_test, y_train, y_test = \
    train_test_split(train1, label1, test_size=0.2, random_state=123)

In [485]:
xg_reg1.fit(X_train,y_train)

XGBRegressor(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=0.3, gamma=0,
       importance_type='gain', learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=10,
       n_jobs=1, nthread=None, objective='reg:squarederror',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=None, subsample=1, verbosity=1)

In [486]:
predicted1 = xg_reg1.predict(X_test)
predicted1 = pd.DataFrame(predicted1)
RMSE = np.sqrt(np.sum(((y_train['label']-predicted1[0])**2)/len(y_train)))
RMSE

41909.14295795053

In [487]:
X_train, X_test, y_train, y_test = \
    train_test_split(train2, label2, test_size=0.2, random_state=123)

In [488]:
xg_reg2 = xgb.XGBRegressor(objective ='reg:squarederror', 
                colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)

In [489]:
xg_reg2.fit(X_train,y_train)

XGBRegressor(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=0.3, gamma=0,
       importance_type='gain', learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=10,
       n_jobs=1, nthread=None, objective='reg:squarederror',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=None, subsample=1, verbosity=1)

In [490]:
predicted2 = xg_reg2.predict(X_test)
predicted2 = pd.DataFrame(predicted2)
RMSE = np.sqrt(np.sum(((y_train['label']-predicted2[0])**2)/len(y_train)))
RMSE

41803.18856169543

In [491]:
X_train, X_test, y_train, y_test = \
    train_test_split(train3, label3, test_size=0.2, random_state=123)

In [492]:
xg_reg3 = xgb.XGBRegressor(objective ='reg:squarederror', 
                colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)

In [482]:
xg_reg3.fit(X_train,y_train)

XGBRegressor(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=0.3, gamma=0,
       importance_type='gain', learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=10,
       n_jobs=1, nthread=None, objective='reg:squarederror',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=None, subsample=1, verbosity=1)

In [483]:
predicted3 = xg_reg3.predict(X_test)
predicted3 = pd.DataFrame(predicted3)
RMSE = np.sqrt(np.sum(((y_train['label']-predicted3[0])**2)/len(y_train)))
RMSE

41892.3043270054

In [456]:
X_train, X_test, y_train, y_test = \
    train_test_split(train4, label4, test_size=0.2, random_state=123)

In [457]:
xg_reg4 = xgb.XGBRegressor(objective ='reg:squarederror', 
                colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)

In [458]:
xg_reg4.fit(X_train,y_train)

XGBRegressor(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=0.3, gamma=0,
       importance_type='gain', learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=10,
       n_jobs=1, nthread=None, objective='reg:squarederror',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=None, subsample=1, verbosity=1)

In [459]:
predicted4 = xg_reg4.predict(X_test)
predicted4 = pd.DataFrame(predicted4)
RMSE = np.sqrt(np.sum(((y_train['label']-predicted4[0])**2)/len(y_train)))
RMSE

41708.97379318869